In [100]:
import numpy as np

class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True 
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
 
            if(grad is None):
                grad = Tensor(np.ones_like(self.data))

            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
                if(self.creation_op == "sub"):
                    self.creators[0].backward(Tensor(self.grad.data), self)
                    self.creators[1].backward(Tensor(self.grad.__neg__().data), self)

                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new , self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)                    
                    
                if(self.creation_op == "mm"):
                    c0 = self.creators[0]
                    c1 = self.creators[1]
                    new = self.grad.mm(c1.transpose())
                    c0.backward(new)
                    new = self.grad.transpose().mm(c0).transpose()
                    c1.backward(new)
                    
                if(self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())

                if("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.expand(dim,
                                                               self.creators[0].data.shape[dim]))

                if("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))
                    
                if(self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__())
                    
                if(self.creation_op == "sigmoid"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (self * (ones - self)))
                
                if(self.creation_op == "tanh"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (ones - (self * self)))
                
                if(self.creation_op == "index_select"):
                    new_grad = np.zeros_like(self.creators[0].data)
                    indices_ = self.index_select_indices.data.flatten()
                    grad_ = grad.data.reshape(len(indices_), -1)
                    for i in range(len(indices_)):
                        new_grad[indices_[i]] += grad_[i]
                    self.creators[0].backward(Tensor(new_grad))
                    
                if(self.creation_op == "cross_entropy"):
                    dx = self.softmax_output - self.target_dist
                    self.creators[0].backward(Tensor(dx))
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op="neg")
        return Tensor(self.data * -1)
    
    def __sub__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="sub")
        return Tensor(self.data - other.data)
    
    def __mul__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)    

    def sum(self, dim):
        if(self.autograd):
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))
    
    def expand(self, dim,copies):

        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_data = self.data.repeat(copies).reshape(list(self.data.shape) + [copies]).transpose(trans_cmd)
        
        if(self.autograd):
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if(self.autograd):
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))
    
    def sigmoid(self):
        if(self.autograd):
            return Tensor(1 / (1 + np.exp(-self.data)),
                          autograd=True,
                          creators=[self],
                          creation_op="sigmoid")
        return Tensor(1 / (1 + np.exp(-self.data)))

    def tanh(self):
        if(self.autograd):
            return Tensor(np.tanh(self.data),
                          autograd=True,
                          creators=[self],
                          creation_op="tanh")
        return Tensor(np.tanh(self.data))
    
    def index_select(self, indices):

        if(self.autograd):
            new = Tensor(self.data[indices.data],
                         autograd=True,
                         creators=[self],
                         creation_op="index_select")
            new.index_select_indices = indices
            return new
        return Tensor(self.data[indices.data])

    def softmax(self):
        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        return softmax_output    
    
    def cross_entropy(self, target_indices):

        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        
        t = target_indices.data.flatten()
        p = softmax_output.reshape(len(t),-1)
        target_dist = np.eye(p.shape[1])[t]
        loss = -(np.log(p) * (target_dist)).sum(1).mean()
    
        if(self.autograd):
            out = Tensor(loss,
                         autograd=True,
                         creators=[self],
                         creation_op="cross_entropy")
            out.softmax_output = softmax_output
            out.target_dist = target_dist
            return out

        return Tensor(loss)
        
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  
    
class Tanh(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.tanh()
    
class Sigmoid(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.sigmoid()

class Layer(object):
    
    def __init__(self):
        self.parameters = list()
        
    def get_parameters(self):
        return self.parameters

    
class SGD(object):
    
    def __init__(self, parameters, alpha=0.1):
        self.parameters = parameters
        self.alpha = alpha
    
    def zero(self):
        for p in self.parameters:
            p.grad.data *= 0
        
    def step(self, zero=True):
        
        for p in self.parameters:
            
            p.data -= p.grad.data * self.alpha
            
            if(zero):
                p.grad.data *= 0


class Linear(Layer):

    def __init__(self, n_inputs, n_outputs):
        super().__init__()
        W = np.random.randn(n_inputs, n_outputs) * np.sqrt(2.0/(n_inputs))
        self.weight = Tensor(W, autograd=True)
        self.bias = Tensor(np.zeros(n_outputs), autograd=True)
        
        self.parameters.append(self.weight)
        self.parameters.append(self.bias)

    def forward(self, input):
        return input.mm(self.weight)+self.bias.expand(0,len(input.data))


class Sequential(Layer):
    
    def __init__(self, layers=list()):
        super().__init__()
        
        self.layers = layers
    
    def add(self, layer):
        self.layers.append(layer)
        
    def forward(self, input):
        for layer in self.layers:
            input = layer.forward(input)
        return input
    
    def get_parameters(self):
        params = list()
        for l in self.layers:
            params += l.get_parameters()
        return params


class Embedding(Layer):
    
    def __init__(self, vocab_size, dim):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.dim = dim
        
        # this random initialiation style is just a convention from word2vec
        self.weight = Tensor((np.random.rand(vocab_size, dim) - 0.5) / dim, autograd=True)
        
        self.parameters.append(self.weight)
    
    def forward(self, input):
        return self.weight.index_select(input)


class Tanh(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.tanh()


class Sigmoid(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.sigmoid()
    

class CrossEntropyLoss(object):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, input, target):
        return input.cross_entropy(target)

    
class RNNCell(Layer):
    
    def __init__(self, n_inputs, n_hidden, n_output, activation='sigmoid'):
        super().__init__()

        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        if(activation == 'sigmoid'):
            self.activation = Sigmoid()
        elif(activation == 'tanh'):
            self.activation == Tanh()
        else:
            raise Exception("Non-linearity not found")

        self.w_ih = Linear(n_inputs, n_hidden)
        self.w_hh = Linear(n_hidden, n_hidden)
        self.w_ho = Linear(n_hidden, n_output)
        
        self.parameters += self.w_ih.get_parameters()
        self.parameters += self.w_hh.get_parameters()
        self.parameters += self.w_ho.get_parameters()        
    
    def forward(self, input, hidden):
        from_prev_hidden = self.w_hh.forward(hidden)
        combined = self.w_ih.forward(input) + from_prev_hidden
        new_hidden = self.activation.forward(combined)
        output = self.w_ho.forward(new_hidden)
        return output, new_hidden
    
    def init_hidden(self, batch_size=1):
        return Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)

# Part 1: RNN Character Language Model

In [132]:
import sys,random,math
from collections import Counter
import numpy as np
import sys

np.random.seed(0)
# dataset from http://karpathy.github.io/2015/05/21/rnn-effectiveness/
f = open('shakespear.txt','r')
raw = f.read()
f.close()

vocab = list(set(raw))
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i
indices = np.array(list(map(lambda x:word2index[x], raw)))

embed = Embedding(vocab_size=len(vocab),dim=512)
model = RNNCell(n_inputs=512, n_hidden=512, n_output=len(vocab))

criterion = CrossEntropyLoss()
optim = SGD(parameters=model.get_parameters() + embed.get_parameters(), alpha=0.05)

batch_size = 32
bptt = 16
n_batches = int((indices.shape[0] / (batch_size)))

trimmed_indices = indices[:n_batches*batch_size]
batched_indices = trimmed_indices.reshape(batch_size, n_batches).transpose()

input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

n_bptt = int(((n_batches-1) / bptt))
input_batches = input_batched_indices[:n_bptt*bptt].reshape(n_bptt,bptt,batch_size)
target_batches = target_batched_indices[:n_bptt*bptt].reshape(n_bptt, bptt, batch_size)


In [143]:
raw[0:5]

'That,'

In [142]:
indices[0:5]

array([30,  4, 37, 42, 52])

In [144]:
batched_indices[0:5]

array([[30, 23,  7, 31, 50,  4, 30,  0,  0, 37, 37,  9, 50,  0, 52, 21,
         0, 61,  4,  7,  9, 37,  0,  0,  0, 33,  0, 33, 33,  0,  0,  8],
       [ 4, 21, 21, 31, 26,  9, 37, 53, 61,  8, 59,  9, 59, 42,  0, 21,
        27, 50, 50, 21, 59,  1, 27, 57,  4,  8, 13, 20,  8, 20, 58,  0],
       [37, 21, 14,  9,  9, 37, 59, 51, 50,  0, 59,  7, 57,  4, 16, 55,
         9, 53, 59, 21, 57,  9, 35,  9,  9, 40, 50, 52,  0, 50, 50, 27],
       [42, 17, 30, 35,  0, 35, 27,  0, 53,  8,  0,  0, 23,  9,  9, 43,
         9, 52, 61, 39, 51,  0, 50,  9, 35, 59, 53, 21, 11, 35, 20, 50],
       [52, 43, 33, 14, 16,  0, 50, 35,  0, 50, 16, 36, 21,  0,  0, 38,
         8,  0,  0, 50, 23, 61, 16, 26, 52, 37, 59, 43, 53,  9,  9, 61]])

In [145]:
input_batches[0][0:5]

array([[30, 23,  7, 31, 50,  4, 30,  0,  0, 37, 37,  9, 50,  0, 52, 21,
         0, 61,  4,  7,  9, 37,  0,  0,  0, 33,  0, 33, 33,  0,  0,  8],
       [ 4, 21, 21, 31, 26,  9, 37, 53, 61,  8, 59,  9, 59, 42,  0, 21,
        27, 50, 50, 21, 59,  1, 27, 57,  4,  8, 13, 20,  8, 20, 58,  0],
       [37, 21, 14,  9,  9, 37, 59, 51, 50,  0, 59,  7, 57,  4, 16, 55,
         9, 53, 59, 21, 57,  9, 35,  9,  9, 40, 50, 52,  0, 50, 50, 27],
       [42, 17, 30, 35,  0, 35, 27,  0, 53,  8,  0,  0, 23,  9,  9, 43,
         9, 52, 61, 39, 51,  0, 50,  9, 35, 59, 53, 21, 11, 35, 20, 50],
       [52, 43, 33, 14, 16,  0, 50, 35,  0, 50, 16, 36, 21,  0,  0, 38,
         8,  0,  0, 50, 23, 61, 16, 26, 52, 37, 59, 43, 53,  9,  9, 61]])

In [146]:
target_batches[0][0:5]

array([[ 4, 21, 21, 31, 26,  9, 37, 53, 61,  8, 59,  9, 59, 42,  0, 21,
        27, 50, 50, 21, 59,  1, 27, 57,  4,  8, 13, 20,  8, 20, 58,  0],
       [37, 21, 14,  9,  9, 37, 59, 51, 50,  0, 59,  7, 57,  4, 16, 55,
         9, 53, 59, 21, 57,  9, 35,  9,  9, 40, 50, 52,  0, 50, 50, 27],
       [42, 17, 30, 35,  0, 35, 27,  0, 53,  8,  0,  0, 23,  9,  9, 43,
         9, 52, 61, 39, 51,  0, 50,  9, 35, 59, 53, 21, 11, 35, 20, 50],
       [52, 43, 33, 14, 16,  0, 50, 35,  0, 50, 16, 36, 21,  0,  0, 38,
         8,  0,  0, 50, 23, 61, 16, 26, 52, 37, 59, 43, 53,  9,  9, 61],
       [ 0, 48, 51, 57, 33,  9, 42,  9, 16,  0,  9,  4, 21, 57, 16,  3,
         0, 20, 27, 51, 21, 50,  0,  0,  0, 16, 57,  8, 31,  0, 52, 51]])

In [ ]:
def train(iterations=400):
    for iter in range(iterations):
        total_loss = 0
        n_loss = 0

        hidden = model.init_hidden(batch_size=batch_size)
        for batch_i in range(len(input_batches)):

            hidden = Tensor(hidden.data, autograd=True)
            loss = None
            losses = list()
            for t in range(bptt):
                input = Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(input=input)
                output, hidden = model.forward(input=rnn_input, hidden=hidden)

                target = Tensor(target_batches[batch_i][t], autograd=True)    
                batch_loss = criterion.forward(output, target)
                losses.append(batch_loss)
                if(t == 0):
                    loss = batch_loss
                else:
                    loss = loss + batch_loss
            for loss in losses:
                ""
            loss.backward()
            optim.step()
            total_loss += loss.data
            log = "\r Iter:" + str(iter)
            log += " - Batch "+str(batch_i+1)+"/"+str(len(input_batches))
            log += " - Loss:" + str(np.exp(total_loss / (batch_i+1)))
            if(batch_i == 0):
                log += " - " + generate_sample(n=70, init_char='\n').replace("\n"," ")
            if(batch_i % 10 == 0 or batch_i-1 == len(input_batches)):
                sys.stdout.write(log)
        optim.alpha *= 0.99
        print()

In [ ]:
train(100)

 Iter:0 - Batch 191/195 - Loss:148.00388828554404                                                                       
 Iter:1 - Batch 191/195 - Loss:20.588816924127116 mhnethet tttttt t t t thett ttth thetttt thetth t tt t tttheth t t ttt
 Iter:2 - Batch 191/195 - Loss:15.282461756020384  h th the the the th the the thet the the the the the the the t the th
 Iter:3 - Batch 191/195 - Loss:13.048394405821716dh th the the the t the t the the the t the th the the the the the the
 Iter:4 - Batch 191/195 - Loss:11.774988723385185dh the the t th the t the the the the the the th th the t the th th th
 Iter:5 - Batch 191/195 - Loss:10.989391522842189dh the the the the the the t the the the the the the t the the the    
 Iter:6 - Batch 191/195 - Loss:10.400423045063138dh the the the t the t the the the the the the the the the the the the
 Iter:7 - Batch 191/195 - Loss:9.9208775761205764da th the the the the the t the the the the the the the the the the th
 Iter:8 - Batch 191/195 - Loss:9.5115

In [134]:
train(100)

 Iter:73 - Batch 133/195 - Loss:1.4278971135025023


In [135]:
train(100)

 Iter:0 - Batch 191/195 - Loss:1.5069461425416464I of ke hadon, and kidains:  r pren  a suco most this thuse are, sir, 
 Iter:1 - Batch 191/195 - Loss:1.4231989098544708 I  hot thi pomimm.  CHENER which thy  weick the man's and w rege, To t
 Iter:2 - Batch 191/195 - Loss:1.3957566042826986 I was aga now, I am the s eakn man's and s as h the seq man's and say 
 Iter:3 - Batch 191/195 - Loss:1.3711573215374844 I wase, and sa day wear and sa day wear and saked  in the chas to the 
 Iter:4 - Batch 191/195 - Loss:1.3063271397979614 I was ageing on the shak to the  eakn mine th every sted and say wend 
 Iter:5 - Batch 191/195 - Loss:1.2782573171839362 I and eytif the stakn my hath somet mest and say we say, sich her, my 
 Iter:6 - Batch 191/195 - Loss:1.2572971763852931 I was   eakn m of the strich  e ha bucthis sty which thy t upof thy to
 Iter:7 - Batch 191/195 - Loss:1.2334132737824353 I  hot this stod  age; And so not agabe sich her, my good lorget the t
 Iter:8 - Batch 191/195 - Loss:1.

KeyboardInterrupt: 

In [140]:
def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor(np.array([word2index[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
        output.data *= 10
        temp_dist = output.softmax()
        temp_dist /= temp_dist.sum()

        m = (temp_dist > np.random.rand()).argmax()
#         m = output.data.argmax()
        c = vocab[m]
        input = Tensor(np.array([m]))
        s += c
    return s
print(generate_sample(n=2000, init_char='\n'))

I war ded abdons would.

CHENRO:
Why, speed no virth to her,
Plirt, goth Plish love,
Befion
 hath if be fe woulds is feally your hir, the confectife to the nightion
As  rent Ron my hath iom
the worse, my goth Plish love,
Befion
Ass untrucerty of my fernight this we namn?

ANG, makes:
That's bond confect fe comes not commonour would be forch the conflill, the confectiffould b off your day, sind it sequns, be gent Rour jus confectife to the nightion
As   poing from your jus  eep of m look o perves, the worse, my goth Plis rept ffough we name:
Thould be good lorges ever word.

DESS:
Where exbinder: if not conflill, the confectife to the nightion
As co move, sir, this we namn?

ANG VINE PAET:
There was courter hower how, my goth Plish lo res
Toures
ever wo formall, have abon, with a good lorges ever word.

DESS:
Where exbinder: if not conflill, the confectife to the nightion
As co mo not?

ANG:
I horses ever with gent may. Thour hot never wear.

PAGTI by him,
And conflill, the confectif yo